In [10]:
# Import required modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os.path
import pickle

import argparse
import torch

import random
import numpy.random
import torch.random

from datetime import datetime

import warnings
warnings.filterwarnings(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Define variables
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

# 2023-05-02
# For Using GPU (CUDA)
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2023-05-02
# For using tensorboard
args.istensorboard = False

args.total_episode = 100
args.learning_rate = 0.1 # Learning rate Alpha
args.boltzmann_tau_initial = 5 # Initial temperature parameter at Boltzmann policy, Tau
args.total_reward = 0
args.epsilon = 0.1

args.max_iteration = 50 # Maximum iteration num. of algorithm, MAX_STEPS
args.replay_batch = 100 # Replay batch size, B
args.nn_update_num = 20 # CNN update number, U: [(1) Constant num. of iteration], (2) Lower limit of loss function value
args.batch_size = 20 # Batch size, N
args.replay_memory = [] # (1) Using list class, (2) Replay memory class by SLM Lab (page. 105~108)

args.gridnum_x = 15
args.gridnum_y = 15
args.gridsize_x = 120 # ft
args.gridsize_y = 120 # ft

args.time_step = 120 # days
args.total_production_time = 600 # days

# 2023-05-02: For reproduction
args.random_seed = 202022673
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

# State: Pressure distribution, Oil saturation

# Action: Well placement (Coordinate of well location)

# Environment: Reservoir simulator

# Reward: NPV at each time segment

args.discount_rate = 0.1 # Used for calculation of NPV
args.discount_factor = 1 # Used for Q-value update

args.simulation_directory = "H:\\Lab_Meeting\\Simulation_Model\\2D_15-by-15_DQN\\2D_15-by-15_DQN_Filetest"

In [18]:
# with open("H:\\Lab_Meeting\\Simulation_Model\\2D_15-by-15_DQN\\2D_15-by-15_DQN_Filetest\\2D_JY_ECLRUN_1.PRT", 'r') as file_read:
#     line = file_read.readline()
#     print(line)
#     while not line.startswith(f"  PRESSURE AT   {120*1}"):
#         line = file_read.readline()
#     for i in range(1,10+1):
#         line = file_read.readline()
#     print(line)
#
#     line_temp = []
#     lines_converted = []
#     for i in range(1, 15+1):
#         # line_temp = [element.strip() for element in line.split()]
#         lines_converted.append([element.strip() for element in line.split()][3::])
#         line = file_read.readline()
#     print(lines_converted)
# print(f"  PRESSURE AT   {120*1}")

################################# Reading Dynamic Data #################################
# idx: simulation file index
# tstep_idx: simulation time step index
# filename: simulation file name
# dynamic_type: dynamic data type to collect ('PRESSURE' or 'SOIL')
def _read_ecl_prt(args, idx: int, tstep_idx: int, filename: str, dynamic_type: str) -> list:
    # Check if dynamic type input is (1) 'PRESSURE' or (2) 'SOIL'
    if not dynamic_type in ['PRESSURE', 'SOIL']:
        print("Assign correct dynamic data output type!")
        return -1

    # File IO
    # 1. Open .PRT file
    with open(args.simulation_directory+'\\'+filename+'_'+str(idx)+'.PRT') as file_read:
        line = file_read.readline()
        if dynamic_type == 'PRESSURE':
            # 2. Fine the location of dynamic data (PRESSURE case)
            while not line.startswith(f"  {dynamic_type} AT   {args.time_step * tstep_idx}"):
                line = file_read.readline()
            # 3. Dynamic data is located at 10th line below the line ["  {dynamic_type} AT   {args.time_step * tstep_idx}"]
            for i in range(1,10+1):
                line = file_read.readline()
            # 4. Collect dynamic data
            lines_converted = []
            for i in range(1, args.gridnum_y+1):
                lines_converted.append([element.strip() for element in line.split()][3::])
                line = file_read.readline()
        elif dynamic_type == 'SOIL':
            # 2. Fine the location of dynamic data (PRESSURE case)
            while not line.startswith(f"  {dynamic_type}     AT   {args.time_step * tstep_idx}"):
                line = file_read.readline()
            # 3. Dynamic data is located at 10th line below the line ["  {dynamic_type} AT   {args.time_step * tstep_idx}"]
            for i in range(1,10+1):
                line = file_read.readline()
            # 4. Collect dynamic data
            lines_converted = []
            for i in range(1, args.gridnum_y+1):
                lines_converted.append([element.strip() for element in line.split()][3::])
                line = file_read.readline()

    return lines_converted

print(_read_ecl_prt(args, 1, 2, '2D_JY_ECLRUN', 'SOIL'))

# print(test)

[['0.73969', '0.73928', '0.73878', '0.73782', '0*73727', '0.73719', '0.73822', '0.73949', '0.73953', '0.73827', '0.73607', '0.73526', '0.73503', '0.73493', '0.73488'], ['0.73917', '0.73898', '0.73861', '0.73791', '0.73729', '0.73723', '0.73748', '0.73773', '0.73741', '0.73642', '0.73570', '0.73522', '0.73505', '0.73495', '0.73488'], ['0.73880', '0.73865', '0.73828', '0.73779', '0.73733', '0.73696', '0.73677', '0.73671', '0.73624', '0.73578', '0.73535', '0.73506', '0.73498', '0.73500', '0.73491'], ['0.73839', '0.73827', '0.73784', '0.73748', '0.73690', '0.73641', '0.73621', '0.73591', '0.73558', '0.73538', '0.73516', '0.73499', '0.73498', '0.73506', '0.73493'], ['0.73795', '0.73768', '0.73729', '0.73694', '0.73635', '0.73595', '0.73576', '0.73542', '0.73523', '0.73512', '0.73506', '0.73502', '0.73513', '0.73509', '0.73489'], ['0.73740', '0.73706', '0.73689', '0.73628', '0.73588', '0.73559', '0.73549', '0.73521', '0.73504', '0.73497', '0.73496', '0.73523', '0*73592', '0.73524', '0.73487'

In [53]:
dynamic_type = 'PRESSURE'

if dynamic_type != ('PRESSURE' or 'SOIL'):
    print("Assign correct dynamic data input type!")